In [50]:
import numpy as np
import anndata
import pandas as pd
import scanpy as sc
import scipy
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt

In [51]:
Path='/media/kusterlab/internal_projects/active/NCI60_Phospho/Final_dataset/'

In [87]:
file='NCI60_FP'
#file='CRC65_FP'
#file='NCI60_GluC'
#file='pNCI60_GluC'

In [88]:
file2='evidence'
file2='proteinGroups'

In [89]:
# For linux lines end with /n, for mac with /r

df=pd.read_csv(f'{Path}/MQ1.5.5.1_{file}_complete/txt/{file2}.txt', sep='\t', lineterminator='\n')

/tmp/ipykernel_755/3484351550.py:3: DtypeWarning: Columns (449,450) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(f'{Path}/MQ1.5.5.1_{file}_complete/txt/{file2}.txt', sep='\t', lineterminator='\n')


In [90]:
cols_kept=[a for a in df.columns if a[:5]=='Inten' and a!='Intensity']

In [91]:
df=df[['Protein IDs']+cols_kept].copy()

In [92]:
df.index=df['Protein IDs']
del df['Protein IDs']

In [93]:
df.columns=[a[10:] for a in df.columns]

In [94]:
def purge_and_capitalize(ccl):
    ccl_str=ccl.replace('-', '')
    ccl_str=ccl_str.replace(' ', '')
    return(ccl_str.upper())

def map_celllines_to_cellosaurus(ccls):
    # Let's remove '-' and spaces and capitalize everything
    D_ccl={}

    for ccl in ccls:
        D_ccl[ccl]=purge_and_capitalize(ccl)
    ccl_red=sorted(set([D_ccl[a] for a in ccls]))
    len(ccl_red)
    
    Path='/home/mgander/Atlantic/data/'
    dfc=pd.read_pickle(f'{Path}/Viability/CCL_dict/Cellosaurus_CCLs.pkl')
    dfc=dfc[dfc['Species']=='NCBI_TaxID=9606; ! Homo sapiens (Human)']

    # Use the synonyms to map to the Cellosaurus ID
    cellosaurus_ids=list(dfc['Cellosaurus_ID'])
    syns=list(dfc['Synonyms'])

    syn_l=[]
    for s in syns:
        if '; ' in s:
            syn_l.append(s.split('; '))
        else:
            syn_l.append(s)
    D_cello={cellosaurus_ids[i]:cellosaurus_ids[i] for i in range(len(cellosaurus_ids))}

    for i in range(len(syn_l)):
        for s in syn_l[i]:
            D_cello[s]=cellosaurus_ids[i]
            
    ccl_red=sorted(set(ccl_red))
    for ccl in ccl_red:
        if ccl in cellosaurus_ids:
            D_cello[ccl]=ccl
            
    D_cello['C106']='C106[HUMANRECTALADENOCARCINOMA]'
    D_cello['CC20']='CC20[HUMANCOLONADENOCARCINOMA]'
    D_cello['NCIADRES']='NCIADRRES'
    D_cello['Hela']='HELA'
    D_cello['HL60(TB)']='HL60'
    D_cello['C125-PM']='x'
    
    keys=list(D_cello.keys())
    # Still missing
    for ccl_red in ccl_red:
        if not ccl_red in keys:
            print(f'{ccl_red}')
    #return([D_ccl[a] for a in ccls])
    return([D_cello[D_ccl[a]] for a in ccls])

In [95]:
df.columns=[a+'_'+b for a,b in zip(map_celllines_to_cellosaurus([a.split('_')[0] for a in df.columns]), 
                                  [a.split('_')[1] for a in df.columns])]

In [96]:
df.to_pickle(f'/home/mgander/Atlantic/data/copied_data/{file}_{file2}.pkl')